In [1]:
import numpy as np
import pandas as pd
import altair as alt
import implicit
import scipy.sparse as sparse
#from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
#import matplotlib as plt

In [2]:
datos = pd.read_csv('C:/Users/gpabl/Documents/git/proyectomat282/datos/datos.csv', low_memory=False) 

In [3]:
datos.head()

,store_nbr,product_nbr,tickets,sales,quantity_dimless
0,0,292337,19,169578,19
1,1,993157,206,490015,301
2,1,475492,103,526162,112
3,2,864448,93,87829,110
4,2,2929,4978,3645122,7172


In [4]:
datos.shape # tamaño del df

(163502, 5)

In [5]:
datos["product_nbr"].nunique()  # numero total de productos 

19629

In [6]:
datos.describe() ## summary

,store_nbr,product_nbr,tickets,sales,quantity_dimless
count,163502.000000,1.635020e+05,163502.000000,1.635020e+05,163502.000000
mean,8.442105,6.140533e+05,515.837378,1.040289e+06,632.889280
std,5.493086,3.293140e+05,1563.652609,2.087246e+07,1932.380182
min,0.000000,7.400000e+01,1.000000,8.400000e+01,1.000000
25%,4.000000,2.996295e+05,31.000000,6.666225e+04,35.000000
50%,9.000000,7.182170e+05,130.000000,2.371200e+05,150.000000
75%,13.000000,9.144350e+05,453.000000,6.952910e+05,537.000000
max,19.000000,1.079974e+06,105100.000000,3.542188e+09,112300.000000


In [7]:
datos["store_nbr"].nunique() ## numero de locales

20

In [8]:
aux1 = (datos.groupby(["store_nbr"])
             .agg(ventas = ("sales","sum"),
                  promedio_ventas = ("sales","mean"),
                  numero_productos = ("product_nbr","count"),
                  venta_maxima = ("sales","max"),
                  qadim_total = ("quantity_dimless","sum"))
             .reset_index()

       )



In [9]:
aux4 = pd.DataFrame(aux1.index)

u1 = aux1.sort_values("ventas",ascending = False)["store_nbr"].reset_index().drop("index",axis=1).rename(columns = {"store_nbr":"ventas"})
u1["index"] = aux4
u2 = aux1.sort_values("promedio_ventas",ascending = False)["store_nbr"].reset_index().rename(columns = {"store_nbr":"promedio_ventas"}).drop("index",axis=1)
u2["index"] = aux4
u3 =  aux1.sort_values("venta_maxima",ascending = False)["store_nbr"].reset_index().rename(columns = {"store_nbr":"venta_maxima"}).drop("index",axis=1)
u3["index"] = aux4
u4 = aux1.sort_values("numero_productos",ascending = False)["store_nbr"].reset_index().rename(columns = {"store_nbr":"numero_productos"}).drop("index",axis=1)
u4["index"] = aux4

aux3 = u1.merge(u2,on="index", how = "left").merge(u3,on="index",how="left").merge(u4,on = "index",how="left").drop("index",axis=1)
aux3

,ventas,promedio_ventas,venta_maxima,numero_productos
0,3,3,11,0
1,0,11,0,10
2,10,10,6,3
3,6,0,15,6
4,5,5,10,5
5,13,6,3,13
6,11,13,2,9
7,15,17,17,2
8,2,15,13,15
9,9,2,9,11


In [10]:
alt.Chart(aux1).mark_bar().encode(
    x = alt.X("store_nbr",title= "store nbr"),
    y = alt.Y(alt.repeat("column"),type = "quantitative")).repeat(
        column = ['ventas','venta_maxima','promedio_ventas','numero_productos']
    )

alt.RepeatChart(...)

### por producto


In [11]:
prod = (datos.groupby("product_nbr")
             .agg(ventas = ("sales","sum"),
                  venta_max = ("sales","max"), 
                  numero_locales = ("store_nbr","count"),
                  tickets_totales = ("tickets","sum"),
                  total_q_adim = ("quantity_dimless","sum"),
                  venta_prom = ("sales","mean"))
              .sort_values("product_nbr")
              .reset_index()
              .assign(precio_prom =  lambda x: x["ventas"]/x["total_q_adim"] )
      )
display(prod.head())

,product_nbr,ventas,venta_max,numero_locales,tickets_totales,total_q_adim,venta_prom,precio_prom
0,74,23540965,2544474,20,31160,40014,1.177048e+06,588.318214
1,106,78375257,10875159,20,50499,66875,3.918763e+06,1171.966460
2,108,796434,773494,3,1039,1429,2.654780e+05,557.336599
3,130,181220,55555,13,610,884,1.394000e+04,205.000000
4,137,5594856,774074,14,5345,6408,3.996326e+05,873.104869


## Score

In [12]:
def score(df):
    v = (datos.sales - datos.sales.min())/(datos.sales.max()-datos.sales.min())
    t = (datos.tickets - datos.tickets.min())/(datos.tickets.max()-datos.tickets.min())
    q = (datos.quantity_dimless - datos.quantity_dimless.min())/(datos.quantity_dimless.max()-datos.quantity_dimless.min()) 
    return 0.5*v + 0.2*t + 0.3*q   

In [13]:
#datos = datos.query("store_nbr == ['0','1','2']")
#datos.head()

In [14]:
#datos['score'] = datos.apply(score, axis=1)

In [15]:
#a = pd.DataFrame(columns = datos["product_nbr"].unique(),index = datos["store_nbr"].unique())

In [16]:
#datos.loc[0]

In [17]:
#datos.loc[0][1,5]


In [18]:
#for i in range(datos.shape[0]):
#    j = int(datos.loc[i][1]) ## product nbr
#    l= int(datos.loc[i][0])
#    k = datos.loc[i][5] ## score nbr
#    a[j][l] = k   


por problemas de memoria por el momento utilizaremos como score simplemente a los tickets.


In [19]:
matrix = datos.set_index(["store_nbr","product_nbr"])["tickets"].unstack("product_nbr").fillna(0).astype(int)
#datos.groupby(["store_nbr","product_nbr"])["tickets"].apply(sum).unstack("product_nbr")

In [20]:
matrix.shape

(20, 19629)

## Descomposicion matricial

In [22]:
sparse_user_item = sparse.csr_matrix(matrix)
sparse_item_user = sparse.csr_matrix(matrix.T)

In [23]:
modelo = implicit.als.AlternatingLeastSquares()


In [24]:
modelo.fit(sparse_item_user,show_progress = False)

In [25]:
recommended = modelo.recommend(2, sparse_user_item,N=10)

In [26]:
recommended

[(6120, 1.0089287),
 (6067, 1.0018529),
 (7345, 0.9982905),
 (1057, 0.9975158),
 (6198, 0.9949728),
 (19598, 0.99264854),
 (11, 0.9923416),
 (6122, 0.9897084),
 (19602, 0.98955053),
 (17624, 0.9863471)]

In [27]:
matrix.tail()

product_nbr,74,106,108,130,137,185,187,195,199,211,...,1069855,1069857,1069859,1069861,1069863,1070859,1070860,1070861,1075696,1079974
store_nbr,,,,,,,,,,,,,,,,,,,,,
15,1437,1450,0,11,521,0,0,0,0,1267,...,346,69,151,0,194,0,0,0,0,379
16,1974,1772,0,15,291,0,0,0,434,1285,...,1193,0,205,0,1096,0,0,0,0,453
17,2942,3500,0,16,0,11,0,0,614,1267,...,174,0,0,24,0,0,0,14,0,479
18,867,1019,0,0,0,172,214,0,82,455,...,16,0,0,46,0,0,0,0,0,196
19,1056,1993,0,0,0,746,502,0,648,509,...,17,0,0,1,23,0,0,0,0,477


In [28]:
#GridSearchCV()